## Proyecto 3 - SDS
### Sara Zavala

In [1]:
##Ataque de extracción 

In [ ]:
import pprint
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras
from sklearn.utils import class_weight
from sklearn import *
from matplotlib.path import Path
from matplotlib.patches import PathPatch
from sklearn.datasets import make_classification
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from art.estimators.classification import KerasClassifier
import tensorflow as tf
from tensorflow.python.framework.ops import disable_eager_execution
from art.attacks.evasion import FastGradientMethod
from art.attacks.evasion import BasicIterativeMethod
from art.defences.trainer import AdversarialTrainer
from art.estimators.classification import TensorFlowV2Classifier


In [ ]:
tf.compat.v1.disable_eager_execution()

In [ ]:
path = ("malimg_paper_dataset_imgs")

In [ ]:
path_images = ImageDataGenerator().flow_from_directory(directory=path, target_size=(128,128), batch_size=10000)

Ataque de evasión 
1. Se debe mostrar un entrenamiento y predicción del modelo original 
2. Utilizando el ataque de evasión Fast Gradient Method genere observaciones falsas 
3. Muestre  la  cantidad  de  observaciones  generadas  que  fueron  clasificadas  correcta  e 
incorrectamente 
4. Explique con detalle los resultados obtenidos. ¿Cómo podría proteger su modelo ante este 
tipo de ataques? 
5. Realce las modificaciones que considere pertinentes para hacer su modelo robusto contra este  tipo  de  ataques  (puede  modificar  el  modelo  original,  así  como  el  dataset  de entrenamiento). Repita los pasos 1 al 3 con el nuevo modelo robusto y demuestre que es menos susceptible a equivocarse con observaciones generadas 

In [ ]:
path_images.class_indices

In [ ]:
data = path_images.class_indices
data2 = list(data.values())
print(len(data2)-1)

In [ ]:
# x, y = next(batches)
images, marks = next(path_images)

In [ ]:
images.shape

In [ ]:
images.size

In [ ]:
marks.size

In [ ]:
print('Images size --> ', images.itemsize)
print('Marks --> ',marks.itemsize)

In [ ]:
images

In [ ]:
marks

In [ ]:
## Entrenamiento y prediccion

In [ ]:
X_train, X_test0, y_train, y_test0 = train_test_split(images, marks, test_size = 0.2)

In [ ]:
## TRAIN

In [ ]:
X_train.shape

In [ ]:
y_train.shape

In [ ]:
## TEST

In [ ]:
X_test0.shape

In [ ]:
y_test0.shape

In [ ]:
## Modelo Original

In [ ]:
model = keras.Sequential()
model.add(keras.layers.Conv2D(32, (5, 5), input_shape=(128, 128, 3), activation='relu', padding='same'))
model.add(keras.layers.MaxPooling2D(2))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Conv2D(64, (3, 3), activation='relu'))
model.add(keras.layers.Conv2D(128, (3, 3), activation='relu'))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(128, activation='relu'))
model.add(keras.layers.Dense(25, activation='softmax'))

In [ ]:
# Compile
# https://faroit.com/keras-docs/2.1.3/models/sequential/
model.compile(loss='categorical_crossentropy',optimizer='SGD',metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
model.layers

In [ ]:
model.fit(X_train, y_train, batch_size=10, validation_data=(X_test0, y_test0), epochs=5)

In [ ]:
## Modelo Robusto

In [ ]:
modelR = keras.Sequential()
modelR.add(keras.layers.Conv2D(32, (5, 5), input_shape=(128, 128, 3), activation='relu', padding='same'))
modelR.add(keras.layers.MaxPooling2D(2))
modelR.add(keras.layers.Dropout(0.2))
modelR.add(keras.layers.BatchNormalization())
modelR.add(keras.layers.Conv2D(64, (3, 3), activation='relu'))
modelR.add(keras.layers.Conv2D(128, (3, 3), activation='relu'))
modelR.add(keras.layers.Flatten())
modelR.add(keras.layers.Dense(1024, activation='relu'))
modelR.add(keras.layers.Dense(25, activation='softmax'))

In [ ]:
modelR.compile(loss='categorical_crossentropy',optimizer='SGD',metrics=['accuracy'])

In [ ]:
modelR.summary()

In [ ]:
modelR.layers

In [ ]:
modelR.fit(X_train, y_train, batch_size=10, validation_data=(X_test0, y_test0), epochs=2)

In [ ]:
print(modelR.optimizer)

In [ ]:
print(modelR.metrics)

In [ ]:
# Comprobacion sobre conjunto de prueba
op = modelR.evaluate(X_test0, y_test0)
print("Exactitud -->", op[1])

In [ ]:
#tf.compat.v1.disable_eager_execution()

In [ ]:
print("")

In [ ]:
model_class = KerasClassifier(model, clip_values=(0, 1), use_logits=False)

In [ ]:
modelR_class = KerasClassifier(modelR, clip_values=(0, 1), use_logits=False)

In [ ]:
cantidad_de_ataques = BasicIterativeMethod(modelR_class, eps=60, eps_step=2, max_iter=40)

In [ ]:
modelR_train = AdversarialTrainer(modelR_class, cantidad_de_ataques, ratio=1.0)

In [ ]:
#Necesito compilarlo antes
modelR_train.fit(X_train, y_train, nb_epochs=2, batch_size=50)

In [ ]:
# Clasificacion de data (con defensa y siendo atacado) de modelo original

In [ ]:
#Gráfica el accuracy del modelo robado